# Práctico Recsim

Adaptado de los tutoriales disponibles en: https://github.com/google-research/recsim por Manuel Cartagena.

In [ ]:
# Install Recsim
!pip install --upgrade --no-cache-dir recsim

## Reinforcement Learning

![RL setup](https://github.com/bamine/recsys-summer-school/raw/12e57cc4fd1cb26164d2beebf3ca29ebe2eab960/notebooks/images/rl-setup.png)


## Tipos de interacción

![texto alternativo](https://github.com/bamine/recsys-summer-school/raw/12e57cc4fd1cb26164d2beebf3ca29ebe2eab960/notebooks/images/organic-bandit.png)

## Importar paquetes necesarios

In [ ]:
import functools
import numpy as np
from gym import spaces
import matplotlib.pyplot as plt
from scipy import stats

from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym
from recsim.simulator import runner_lib

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

## Recsim
![RecSim implementation](https://github.com/google-research/recsim/blob/master/recsim/colab/figures/simulator_implemented.png?raw=true)

# Resumen

Un paso en la simulación de Recsim consiste en:


1.   La Base de Datos de Documentos (items) provee un corpus de *D* documentos al recomendador.
2.   El recomendador observa los *D* documentos (y sus features) junto a las respuestas del usuario para la última recomendación. Luego hace una selección ordenada de *k* documentos para presentárselos al usuario.
3.   El usuario examina la lista y escoge a lo más un documento (no escoger uno es una opción). Esto genera una transición del estado del usuario. Finalmente el usuario emite una observación del documento, que en la siguiente iteración el recomendador podrá ver.

# Escenario de la simulación: Videos de Memes vs Educativos

Los documentos de nuestro corpus corresponderan a items (en este caso videos) que se caracterizan por su grado de educativo o de meme. Documentos "meme" generan alto compromiso (**engagement**), pero _hipotéticamente_ el consumo a largo plazo de estos documentos lleva a disminuir la satisfacción del usuario. Por otro lado, documentus educativos generan relativamente bajo engagement, pero su consumo conlleva a una mayor satisfacción a largo plazo. Modelaremos esta propiedad de los documentos como una feature continua que puede tomar valores entre [0,1], le llamaremos Educativeness-scale. Un documento con score 1 es totalmente educativo, mientras que un document con score 0 es totalmente meme.

El estado latente del usuario consiste en una variable de dimensión 1 llamada *satisfacción*. Cada vez que consume un documento "educativo", esta variable tiende a incrementar, y opuestamente, un documento meme tiende a disminuir la satisfacción.

Al consumir un documento, el usuario emite una medida estocástica del engagement (tiempo que ve el video) sobre el documento. Este valor es proporcional a la satisfacción del usuario e inversamente proporcional a la educatividad del documento en cuestión.

Por lo tanto, el objetivo es encontrar el mix óptimo de documentos para mantener el engagement del usuario por un período largo de tiempo.

## Document

### Model

Clase que define los documentos, LTS es una abreviación de Long Term Satisfaction

In [ ]:
class LTSDocument(document.AbstractDocument):
  def __init__(self, doc_id, educativeness, cluster_id):
    self.educativeness = educativeness
    self.cluster_id = cluster_id
    # doc_id es un ID unico para el documento
    super(LTSDocument, self).__init__(doc_id)

  NUM_CLUSTERS = 4

  # Una observación son los valores públicos del documento
  def create_observation(self):
    return {'educativeness': np.array(self.educativeness), 'cluster_id': self.cluster_id}

  # El espacio de la observación utiliza la el estándar del gym de OpenAI: https://gym.openai.com/docs/#spaces
  @classmethod
  def observation_space(self):
    return spaces.Dict({
      'educativeness': spaces.Box(shape=(1,), dtype=np.float32, low=0.0, high=1.0),
      'cluster_id': spaces.Discrete(self.NUM_CLUSTERS)
    })
  
  # Método para definir cómo se imprime un documento
  def __str__(self):
    return "Document {} from cluster {} with educativeness {}.".format(self._doc_id, self.cluster_id, self.educativeness)

### Sampler

Un Sampler es una clase que creará una instancia del objeto en cuestión, en este caso para los documentos

In [ ]:
class LTSDocumentSampler(document.AbstractDocumentSampler):
  def __init__(self, doc_ctor=LTSDocument, **kwargs):
    super(LTSDocumentSampler, self).__init__(doc_ctor, **kwargs)
    self._doc_count = 0

  def sample_document(self):
    doc_features = {}
    doc_features['doc_id'] = self._doc_count
    doc_features['educativeness'] = self._rng.random_sample()
    doc_features['cluster_id'] = self._rng.choice(self._doc_ctor.NUM_CLUSTERS)
    self._doc_count += 1
    return self._doc_ctor(**doc_features)

Ejemplo de sampleo de documentos

In [ ]:
sampler = LTSDocumentSampler()
for i in range(5): print(sampler.sample_document())
d = sampler.sample_document()
print("Documents have observation space:", d.observation_space(), "\n"
      "An example realization is: ", d.create_observation())

## User

El modelo de usuario para este tutorial es:
* Cada usuario tiene una feature llamada net educativeness exposure ($\text{nee}_t$), y satisfacción ($\text{sat}_t$). Están relacionadas mediante una función logística para reflejar que la satisfacción no puede no tener un límite.
$$\text{sat}_t = \sigma(\tau\cdot\text{nee}_t),$$
donde $\tau$ es un parámetro de sensitividad específico por usuario.
* Dado un slate $S$, el usuario escoge un item basado en un modelo de decisión multinomial con la educativeness como feature: $p(\text{usuario escoja }d_i \text{ del slate }S) \sim e^{1-\mathrm{educativeness}(d_i)}$
* Una vez el usuario escoge un documento, la net educativeness exposure evoluciona de la manera:
$$\text{nee}_{t+1} = \beta \cdot \text{nee}_t + 2(k_d - 1/2) + {\cal N}(0, \eta),$$
donde $\beta$ es un factor específico por usuario que llamaremos memory discount (factor de olvido), $k_d$ es la educativeness del documento escogido y $\eta$ es ruido proveniente de una distribución normal que llamaremos innovación (innovation).
* Finalmente, el usuario interactúa con el contenido escogido por $s_d$ segundos, donde $s_d$ es sacado de alguna distribución
$$s_d\sim\log{\cal N}(k_d\mu_k + (1-k_d)\mu_c, k_d\sigma_k + (1-k_d)\sigma_c),$$
por ejemplo, una distribución log-normal con interpolando linealmente entre una respuesta puramente educativa $(\mu_k, \sigma_k)$ y una respuesta puramente meme $(\mu_c, \sigma_c)$.

De acuerdo a esto, el estado de un usuario está definido por la tupla $(\text{sat}, \tau, \beta, \eta, \mu_k, \sigma_k, \mu_c, \sigma_c).$ La satisfacción es la única variable dinámica del estado.



### State

Esta clase maneja el estado del usuario durante una simulación, tanto las variables públicas como privadas de este durante el tiempo.

In [ ]:
class LTSUserState(user.AbstractUserState):
  def __init__(self, memory_discount, sensitivity, innovation_stddev,
               meme_mean, meme_stddev, educ_mean, educ_stddev,
               net_educativeness_exposure, time_budget, observation_noise_stddev=0.1
              ):
    ## Transition model parameters
    self.memory_discount = memory_discount
    self.sensitivity = sensitivity
    self.innovation_stddev = innovation_stddev

    ## Engagement parameters
    self.meme_mean = meme_mean
    self.meme_stddev = meme_stddev
    self.educ_mean = educ_mean
    self.educ_stddev = educ_stddev

    ## State variables
    self.net_educativeness_exposure = net_educativeness_exposure
    self.satisfaction = 1 / (1 + np.exp(-sensitivity * net_educativeness_exposure))
    self.time_budget = time_budget

    # Noise
    self._observation_noise = observation_noise_stddev

  # Al igual que con los documentos, se retorna la observación del estado del usuario, en este caso lo único público es su satisfacción
  def create_observation(self):
    """User's state is not observable."""
    clip_low, clip_high = (-1.0 / (1.0 * self._observation_noise),
                           1.0 / (1.0 * self._observation_noise))
    noise = stats.truncnorm(
        clip_low, clip_high, loc=0.0, scale=self._observation_noise).rvs()
    noisy_sat = self.satisfaction + noise
    return np.array([noisy_sat,])

  # También hay que definir el espacio de las variables que se retornen de una observación
  @staticmethod
  def observation_space():
    return spaces.Box(shape=(1,), dtype=np.float32, low=-2.0, high=2.0)
  
  # Función de score para usar en el modelo de selección del usuario: en este caso el usuario tenderá a elegir más contenido de memes
  def score_document(self, doc_obs):
    return 1 - doc_obs['educativeness']


### Sampler

Clase que sampleará los usuarios para la simulación, en este caso hay muchos parámetros que quedarán hardcodeados, pero se puede hacer dinámico.

In [ ]:
class LTSStaticUserSampler(user.AbstractUserSampler):
  _state_parameters = None

  def __init__(self,
               user_ctor=LTSUserState,
               memory_discount=0.9,
               sensitivity=0.01,
               innovation_stddev=0.05,
               meme_mean=5.0,
               meme_stddev=1.0,
               educ_mean=4.0,
               educ_stddev=1.0,
               time_budget=60,
               **kwargs):
    self._state_parameters = {'memory_discount': memory_discount,
                              'sensitivity': sensitivity,
                              'innovation_stddev': innovation_stddev,
                              'meme_mean': meme_mean,
                              'meme_stddev': meme_stddev,
                              'educ_mean': educ_mean,
                              'educ_stddev': educ_stddev,
                              'time_budget': time_budget
                             }
    super(LTSStaticUserSampler, self).__init__(user_ctor, **kwargs)

  def sample_user(self):
    starting_nee = ((self._rng.random_sample() - .5) *
                    (1 / (1.0 - self._state_parameters['memory_discount'])))
    self._state_parameters['net_educativeness_exposure'] = starting_nee
    return self._user_ctor(**self._state_parameters)

### Response

Clase que define como es la respuesta de un usuario al interactuar con un documento.

In [ ]:
class LTSResponse(user.AbstractResponse):
  # The maximum degree of engagement.
  MAX_ENGAGEMENT_MAGNITUDE = 100.0

  def __init__(self, cluster_id, clicked=False, engagement=0.0):
    self.clicked = clicked
    self.engagement = engagement
    self.cluster_id = cluster_id

  # Se crea la observación: si dió o no click, cuanto tiempo vió el item y a que cluster pertenece.
  def create_observation(self):
    return {'click': int(self.clicked),
            'engagement': np.array(self.engagement),
            'cluster_id': self.cluster_id}

  # Se define el espacio de estas variables
  @classmethod
  def response_space(cls):
    # `engagement` feature range is [0, MAX_ENGAGEMENT_MAGNITUDE]
    return spaces.Dict({
        'click':
            spaces.Discrete(2),
        'engagement':
            spaces.Box(
                low=0.0,
                high=cls.MAX_ENGAGEMENT_MAGNITUDE,
                shape=tuple(),
                dtype=np.float32),
        'cluster_id':
            spaces.Discrete(4)
    })

### Model

Finalmente se define el modelo del usuario, el cual se compone por las clases definidas anteriormente

In [ ]:
class LTSUserModel(user.AbstractUserModel):
    def __init__(self, slate_size, seed=0):
        super(LTSUserModel, self).__init__(LTSResponse, LTSStaticUserSampler(LTSUserState, seed=seed), slate_size)
        self.choice_model = MultinomialLogitChoiceModel({})
    
    def is_terminal(self):
        # Retorna un boolean si la sesión se terminó, ya que el user tiene una variable de tiempo disponible (time_budget)
        return self._user_state.time_budget <= 0

    def simulate_response(self, slate_documents):
        # Lista con respuestas vacías a partir del slate
        responses = [self._response_model_ctor(d.cluster_id) for d in slate_documents]
        # Se usa el choice_model del user para saber a qué documento le hace click
        self.choice_model.score_documents(self._user_state,
                                          [doc.create_observation() for doc in slate_documents])
        scores = self.choice_model.scores
        selected_index = self.choice_model.choose_item()
        # Se genera la respuesta para el item que se clickeó
        self.generate_response(slate_documents[selected_index],
                               responses[selected_index])
        return responses

    def generate_response(self, doc, response):
        response.clicked = True
        # Se interpola linealmente entre meme y educativo
        engagement_loc = (doc.educativeness * self._user_state.meme_mean + (1 - doc.educativeness) * self._user_state.educ_mean)
        engagement_loc *= self._user_state.satisfaction
        engagement_scale = (doc.educativeness * self._user_state.meme_stddev + ((1 - doc.educativeness) * self._user_state.educ_stddev))
        log_engagement = np.random.normal(loc=engagement_loc,
                                          scale=engagement_scale)
        response.engagement = np.exp(log_engagement)

    # Función que hace update del estado del usuario
    def update_state(self, slate_documents, responses):
        for doc, response in zip(slate_documents, responses):
            if response.clicked:
                innovation = np.random.normal(scale=self._user_state.innovation_stddev)
                net_educativeness_exposure = (self._user_state.memory_discount * self._user_state.net_educativeness_exposure - 2.0 * (doc.educativeness - 0.5) + innovation)
                self._user_state.net_educativeness_exposure = net_educativeness_exposure
                satisfaction = 1 / (1.0 + np.exp(-self._user_state.sensitivity * net_educativeness_exposure))
                self._user_state.satisfaction = satisfaction
                self._user_state.time_budget -= 1
                return


## Crear environment: parámetros
* *slate_size*: Tamaño del set de items a presentar al usuario.
* *num_candidates*: número de documentos presentes en la base de datos en cualquier momento de la simulación.
* *resample_documents*: especifica si se vuelven a samplear los documentos desde la base de datos entre episodios de la simulación.

In [ ]:
slate_size = 3
num_candidates = 10
ltsenv = environment.Environment(
    LTSUserModel(slate_size),
    LTSDocumentSampler(),
    num_candidates,
    slate_size,
    resample_documents=True)


### Parámetro a optimizar: Engagement

In [ ]:
def clicked_engagement_reward(responses):
    reward = 0.0
    for response in responses:
        if response.clicked:
            reward += response.engagement
    return reward

In [ ]:
# Instanciar environment
lts_gym_env = recsim_gym.RecSimGymEnv(ltsenv, clicked_engagement_reward)

In [ ]:
observation_0 = lts_gym_env.reset()
print('Observation 0')
print('Available documents')
doc_strings = ['doc_id ' + key + " educativeness " + str(value) for key, value
               in observation_0['doc'].items()]
print('\n'.join(doc_strings))
print('Noisy user state observation')
print(observation_0['user'])
# "Agente" recomienda los primeros 3 documentos
recommendation_slate_0 = [0, 1, 2]
observation_1, reward, done, _ = lts_gym_env.step(recommendation_slate_0)
print('Observation 1')
print('Available documents')
doc_strings = ['doc_id ' + key + " educativeness " + str(value) for key, value
               in observation_1['doc'].items()]
print('\n'.join(doc_strings))
rsp_strings = [str(response) for response in observation_1['response']]
print('User responses to documents in the slate')
print('\n'.join(rsp_strings))
print('Noisy user state observation')
print(observation_1['user'])

## Agent

In [ ]:
from recsim import agent
from recsim.agents.layers.abstract_click_bandit import AbstractClickBanditLayer
from recsim.agents.layers.cluster_click_statistics import ClusterClickStatsLayer

Crearemos un agente simple que ordene los documentos de un tópico de acuerdo a su educativeness

In [ ]:
class GreedyClusterAgent(agent.AbstractEpisodicRecommenderAgent):
  def __init__(self, observation_space, action_space, cluster_id, pro_educ, **kwargs):
    del observation_space
    super(GreedyClusterAgent, self).__init__(action_space)
    self._cluster_id = cluster_id
    self.pro_educ = pro_educ

  def step(self, reward, observation):
    del reward
    my_docs = []
    my_doc_educativeness = []
    for i, doc in enumerate(observation['doc'].values()):
      if doc['cluster_id'] == self._cluster_id:
        my_docs.append(i)
        my_doc_educativeness.append(doc['educativeness'])
    if not bool(my_docs):
      return []
    # Agregamos esta variable booleana para determinar si ordena los documentos de mayor a menor o al revés (algunos agentes preferirán recomendar los memes primero)
    if self.pro_educ:
        sorted_indices = np.argsort(my_doc_educativeness)[::-1]
    else:
        sorted_indices = np.argsort(my_doc_educativeness)
    return list(np.array(my_docs)[sorted_indices])


In [ ]:
# Obtenemos el número de tópicos disponibles
num_topics = LTSDocument.observation_space()['cluster_id'].n
# Creamos un agente para cada tópico
base_agent_ctors = [functools.partial(GreedyClusterAgent, cluster_id=i, pro_educ=np.random.choice([True, False], 1)[0]) for i in range(num_topics)]

In [ ]:
# Recsim posee clases que se pueden usar como "capas" en keras o pytorch, aquí usamos AbstractBanditLayer que recibe un conjunto de agents que trata como arms
bandit_ctor = functools.partial(AbstractClickBanditLayer, arm_base_agent_ctors=base_agent_ctors)
# Otra capa que se puede usar es ClusterClickStatsLayer la cual le pasa información del número de clicks que ha hecho el usuario a cada cluster
cluster_bandit = ClusterClickStatsLayer(bandit_ctor,
                                        lts_gym_env.observation_space,
                                        lts_gym_env.action_space)

Ejemplo de recomendación hecho por este cluster de bandits

In [ ]:
observation0 = lts_gym_env.reset()
slate = cluster_bandit.begin_episode(observation0)
print("Cluster bandit slate 0:")
doc_list = list(observation0['doc'].values())
for doc_position in slate:
    print(doc_list[doc_position])

Agregaremos una función que toma los parámetros de la simulación y crea nuestro agente

In [ ]:
def create_agent(sess, environment, eval_mode, summary_writer=None):
    kwargs = {
        'observation_space': environment.observation_space,
        'action_space': environment.action_space,
        'summary_writer': summary_writer,
        'eval_mode': eval_mode,
    }
    return ClusterClickStatsLayer(bandit_ctor, **kwargs)

### Entrenamiento

In [ ]:
tmp_base_dir = '/tmp/recsim/'
lts_gym_env.reset()
runner = runner_lib.TrainRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=lts_gym_env,
    episode_log_file="",
    max_training_steps=100,
    num_iterations=20)
runner.run_experiment()

## Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=/tmp/recsim/

# Actividades

### Actividad 1:

Entrene por más episodios y describa lo que está ocurriendo con el agente y el usuario.

### Actividad 2

Explique con sus palabras cuál es la principal ventaja de utilizar una librería como recsim o recogym para Reinforcement Learning

### Actividad 3

¿Cómo se podría mejorar la forma de modelar al usuario?